In [1]:
import numpy as np
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

In [3]:
batch_size = 32 
img_height = 224
img_width = 224 

In [4]:
validation_ds = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred', 
            image_size=(img_height,img_width), batch_size=None, seed=42)

Found 50947 files belonging to 7 classes.


In [5]:
training_ds = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=None, seed=42)

Found 399839 files belonging to 7 classes.


In [6]:
def preprocess(image, label):
    final_image = keras.applications.xception.preprocess_input(image)
    return final_image, label

In [7]:
training_s = training_ds.map(preprocess).batch(batch_size).prefetch(1)
validation_s = validation_ds.map(preprocess).batch(batch_size).prefetch(1)

In [8]:
base_model = keras.applications.Xception(input_shape=(224,224,3), include_top = False, weights=None)

In [9]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [10]:
opt = keras.optimizers.Adam(learning_rate=1e-4)

In [11]:
es = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

In [12]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_s, validation_data=validation_s, epochs=10, callbacks=[es])

Epoch 1/10
12495/12495 [==============================] - 7429s 594ms/step - loss: 0.9179 - accuracy: 0.6561 - val_loss: 0.8704 - val_accuracy: 0.6802
Epoch 2/10
12495/12495 [==============================] - 7482s 599ms/step - loss: 0.6897 - accuracy: 0.7464 - val_loss: 0.6800 - val_accuracy: 0.7520
Epoch 3/10
12495/12495 [==============================] - 7429s 595ms/step - loss: 0.5539 - accuracy: 0.7971 - val_loss: 0.7473 - val_accuracy: 0.7449
Epoch 4/10
12495/12495 [==============================] - 7403s 592ms/step - loss: 0.4397 - accuracy: 0.8399 - val_loss: 0.8098 - val_accuracy: 0.7442


In [14]:
model.evaluate(training_s)

12495/12495 [==============================] - 1554s 124ms/step - loss: 0.5713 - accuracy: 0.7923


[0.5712570548057556, 0.7923364043235779]

In [15]:
model.evaluate(validation_s)

1593/1593 [==============================] - 192s 120ms/step - loss: 0.6800 - accuracy: 0.7520


[0.6800285577774048, 0.751997172832489]

In [16]:
model.save("0902_Xception_no_weight.h5")